# Prerrequisitos

In [ ]:
import numpy as np
!pip install -q git+https://github.com/tensorflow/docs
!pip install heartpy
!pip install git+https://www.github.com/keras-team/keras-contrib.git
import random
from __future__ import division #para que la division de enteros de un float y no el floor de la division
import skimage
import scipy
from skimage import morphology
import pandas as pd
import csv
from numpy import *
import heartpy as hp
from scipy.signal import butter, lfilter
from numpy import cos, sin, pi, absolute, arange
from scipy.signal import kaiserord, lfilter, firwin, freqz
from pylab import figure, clf, plot, xlabel, ylabel, xlim, ylim, title, grid, axes, show
import matplotlib.pyplot as plt
from scipy.signal import freqz
from scipy.signal import convolve as sig_convolve
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
# import peakutils
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import os
!pip install heartpy
!pip install wfdb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from google.colab import drive
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Flatten, Dropout,Activation,BatchNormalization, Convolution2D, MaxPooling2D,Conv1D,MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout,Activation,BatchNormalization, Convolution2D, MaxPooling2D, Input, Add
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# from keras.optimizers import Adam,SGD,RMSprop
from keras_contrib.callbacks import CyclicLR
# from keras.losses import huber_loss
import pickle
import csv
from keras.models import Model
!pip install git+https://www.github.com/keras-team/keras-contrib.git

# from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# from keras.optimizers import Adam,SGD
from keras_contrib.callbacks import CyclicLR
# from keras.losses import huber_loss
import tensorflow as tf

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from matplotlib import pyplot

from sklearn.model_selection import KFold
# from keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

drive.mount('/content/drive',force_remount=True)

     |████████████████████████████████| 1.0 MB 5.3 MB/s 
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-iw7ulco6
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-iw7ulco6
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=1c32360787d6d659b78dc915b88fa7be4e93cfa38608ef7192a91d4910e061b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-6g4fg10o/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib
     |████████████████████████████████| 137 kB 5.1 MB/s 
     |████████████████████████████████| 10.3 MB 43.5 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-znzebphk
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-znzebphk
Mounted at /content/drive


## Funciones, imports varios

In [ ]:
def classify_file_size(path_drive):
  peso = os.path.getsize(path_drive)
  cantidad_bytes_30s = 12+125*(30*125) #peso en bytes de un archivo en csv con 30s

  if(peso>= cantidad_bytes_30s):
    return 1
  else:
    return 0

def kurtosis(ppg):
  N = len(ppg)
  suma = 0
  mu = np.mean(ppg)
  s = np.std(ppg)

  for i in range(N):
    suma = suma + (ppg[i]-mu)**4

  suma = suma/(N*(s)**4)
  return suma
def kurtosis_fft(ppg):
  ppg_fft=abs(np.fft.fft(ppg)[13:625])
  N = len(ppg_fft)
  suma = 0
  mu = np.mean(ppg_fft)
  s = np.std(ppg_fft)

  for i in range(N):
    suma = suma + (ppg_fft[i]-mu)**4

  suma = suma/(N*(s)**4)
  return suma

def skew_libro(ppg):
  N=len(ppg)
  s=np.std(ppg)
  mu=np.mean(ppg)
  suma=0
  for i in range(N):
    suma=suma+((ppg[i]-(mu))**3)
  suma=suma/(N*(s**3))
  return suma

def sacar_corrimiento(signal,signal_wd):
  rolling_mean = signal_wd.get('rolling_mean')

  sin_corrimiento = signal-rolling_mean
  return sin_corrimiento

def nans(signal):
  for i, item in enumerate(signal):
    if (isnan(item)):
      signal[i] = 0
    
  return signal
  
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs #Nyquist frequeny is half the sampling frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a
def butter_lowpass_filter(data, cutoff, fs, order):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def min_cl(pos_min,val_min,media):

  minimos_provisorios = []
  for i in range(len(pos_min)):
    if(val_min[i]>= 0.85*media):
      minimos_provisorios.append(pos_min[i])
      
  minimos_reales = list(minimos_provisorios)

  for k in range(1,len(minimos_provisorios)):    
    actual = minimos_provisorios[k-1]
    siguiente = minimos_provisorios[k]

    diferencia = siguiente-actual
    if(diferencia<42):
      minimos_reales.remove(minimos_provisorios[k])

  return minimos_reales

def max_cl(pos_max,hr,media):
  fs=125
  peaks_sist=[]
  for peak in pos_max:
    if (hr[peak]>0.85*media):
      peaks_sist.append(peak)

  peaks_sist_cl=[]
  for i in range(len(peaks_sist)-1):
    samples=peaks_sist[i+1]-peaks_sist[i]
    bpm=60/(samples/fs)
    if ((bpm<180)&(bpm>30)):
      peaks_sist_cl.append(peaks_sist[i])

  return peaks_sist_cl

def min_cl(pos_min,val_min,media):

  minimos_provisorios = []
  for i in range(len(pos_min)):
    if(val_min[i]>= 0.85*media):
      minimos_provisorios.append(pos_min[i])
      
  minimos_reales = list(minimos_provisorios)

  for k in range(1,len(minimos_provisorios)):    
    actual = minimos_provisorios[k-1]
    siguiente = minimos_provisorios[k]

    diferencia = siguiente-actual
    if(diferencia<42):
      minimos_reales.remove(minimos_provisorios[k])

  return minimos_reales

def picos(signal):

  w,m=hp.process(signal,125)
  pos_maximos = w.get('peaklist')
  hr=w.get('hr')
  media=np.mean(signal[pos_maximos])

  maximos=max_cl(pos_maximos,hr,media) 
  val_maximos = signal[maximos]

  inver = -signal
  winv, minv = hp.process(inver, 125)
  pos_minimos = winv.get('peaklist')
  val_minimos = inver[pos_minimos]

  media=np.mean(val_minimos)
  minimos=min_cl(pos_minimos,val_minimos,media)
  val_minimos=signal[minimos]

  return [maximos,val_maximos, minimos,pos_minimos]

def cortar_pulsos_ppg(signal):

  inver=-signal
  winv,minv=hp.process(inver,125)
  pos_minimos = winv.get('peaklist')
  val_minimos=inver[pos_minimos]
  media=np.mean(val_minimos)
  min_reales=min_cl(pos_minimos,val_minimos,media)

  pulse=[]
  peaks=[]
  for i in range(len(min_reales)-1):
      pulse.append(signal[min_reales[i]:min_reales[i+1]])#tengo un pulso entero entree 2 minimos
      pico=np.amax(pulse[i])
      pos=list(pulse[i]).index(pico)
      peaks.append(pos)
 
  return pulse,min_reales, peaks

def cortar_pulsos_abp(signal):

  _,_,_,minimos = picos(signal)
  min_reales = minimos #porque la funcion picos ya checkea que sean reales

  pulse=[]
  peaks=[]
  for i in range(len(min_reales)-1):
      pulse.append(signal[min_reales[i]:min_reales[i+1]])#tengo un pulso entero entree 2 minimos
      pico=np.amax(pulse[i])
      pos=list(pulse[i]).index(pico)
      peaks.append(pos)
  i=0
  return pulse,min_reales,peaks

def sacar_posiciones_pulsos(minims,pos_i):
  #devuelve las posiciones inicial y final de los pulsos del segmento dado REALES dentro del archivo
  posiciones = []
  for i in range(len(minims)-1):
    posiciones.append([minims[i]+pos_i, minims[i+1]+pos_i])
  return posiciones

def centro_pulso(posiciones):
  center = (posiciones[1]-posiciones[0])/2
  return center

def encontrar_pulso_central(pos_pulso_ppg, pos_pulsos_abp):
  pos_i_ppg = pos_pulso_ppg[0]
  pos_f_ppg = pos_pulso_ppg[1]

  center_ppg = centro_pulso(pos_pulso_ppg) + pos_i_ppg

  distancia_minima = 1000000
  pos_pulso_central = 0

  for i in range(len(pos_pulsos_abp)):
    pulso_abp_actual = pos_pulsos_abp[i]
    pos_i_abp = pulso_abp_actual[0]
    pos_f_abp = pulso_abp_actual[1] 
    center_abp = centro_pulso(pulso_abp_actual) + pos_i_abp

    distancia = abs(center_ppg-center_abp)

    if(distancia<distancia_minima):
      distancia_minima = distancia
      pos_pulso_central = i

    else:
      if((distancia == distancia_minima) and (pos_i_abp >= pos_i_ppg and pos_f_abp <= pos_f_ppg)): #si la distnacia es igual a la minima encontrada y el pulso se encuentra contenido totalmente
          pos_pulso_central = i

  return pos_pulso_central, distancia_minima

def linkear_pulsos(signal_ppg, pos_i_ppg, signal_abp,pos_i_abp):
  #asumo como que en signal ya le estoy pasando los segmentos correspondientes pero podria cambiarse
  #tambien necesito las posiciones iniciales de los segmentos en el archivo real

  _, minimos_abp,_ = cortar_pulsos_abp(signal_abp)
  pulsos_ppg, minimos_ppg,_ = cortar_pulsos_ppg(signal_ppg)
  Nppg = len(pulsos_ppg)
  posiciones_pulsos_abp = sacar_posiciones_pulsos(minimos_abp, pos_i_abp)
  posiciones_pulsos_ppg = sacar_posiciones_pulsos(minimos_ppg, pos_i_ppg)
  numero_pulsos_correspondientes = []
  posiciones_pulsos_correspondientes = []

  for i in range(Nppg):
  
    pos_ppg = posiciones_pulsos_ppg[i] #agarra la primera y ultima posicion del pulso i de ppg
    pos_central,_ = encontrar_pulso_central(pos_ppg, posiciones_pulsos_abp)
    numero_pulsos_correspondientes.append([i, pos_central])
    posiciones_pulsos_correspondientes.append([pos_ppg,posiciones_pulsos_abp[pos_central]])

  return numero_pulsos_correspondientes, posiciones_pulsos_correspondientes #devuelvo numero de pulsos correspondientes de a pares [x,y] y las posiciones de esos pulsos dentro de los vectores

def obtener_BP(signal_ppg, signal_abp, pos_i_ppg, pos_i_abp):
  ppg_nan=nans(signal_ppg)
  
  wd,m=hp.process(ppg_nan,125)
  ppg_sin_corr=sacar_corrimiento(ppg_nan,wd)
 
  cutoff = 8
  ppg_filtrada = butter_lowpass_filter(ppg_sin_corr, cutoff, fs, order=4)
  pulsos_ppg,_,_ = cortar_pulsos_ppg(ppg_filtrada)
  pulsos_abp,_,_ = cortar_pulsos_abp(signal_abp)
  presiones = calculo_ABP(signal_abp)
  pos_pulsos,pos_reales_pulsos = linkear_pulsos(ppg_filtrada,pos_i_ppg, signal_abp, pos_i_abp)

  results = []
  # results_array = np.zeros([len(pulsos_ppg), 3])
  j = 0 
  
  for i in range(len(pulsos_ppg)):
    pulse_abp = pos_pulsos[i][1]
    pulse_ppg = pos_pulsos[i][0]

    if(pulse_abp == 0):
      j = i+1
      results.append([pos_pulsos[i],pos_reales_pulsos[i],presiones[pulse_abp]])
    else:
      results.append([pos_pulsos[i],pos_reales_pulsos[i],(presiones[pulse_abp]+presiones[pulse_abp-1])/2])

  return results

def plotear_pulsos(signal_ppg, signal_abp, pos_i_ppg, pos_i_abp):
  ppg_1_nan=nans(signal_ppg)
  wd_1,m_1=hp.process(ppg_1_nan,125)
  ppg_1_sin_corr=sacar_corrimiento(ppg_1,wd_1)
  cutoff = 8
  y1 = butter_lowpass_filter(ppg_1_sin_corr, cutoff, fs, order=4)

  j,k = linkear_pulsos(y1,s1_i, abp_1, b1_i)
  
  _, minimos_abp,_ = cortar_pulsos_abp(signal_abp)
  pulsos_ppg, minimos_ppg,_ = cortar_pulsos_ppg(y1)

  posiciones_pulsos_abp = sacar_posiciones_pulsos(minimos_abp, pos_i_abp)
  posiciones_pulsos_ppg = sacar_posiciones_pulsos(minimos_ppg, pos_i_ppg)
  print(posiciones_pulsos_ppg)
  plt.figure(1)
  plt.figure(2)
  for i in range(len(j)):
    num_ppg = j[i][0]
    num_abp = j[i][1]

    # print(i)
    # print(num_ppg)

    p1 = posiciones_pulsos_ppg[num_ppg][0] - pos_i_ppg
    p2 = posiciones_pulsos_ppg[num_ppg][1] - pos_i_ppg
    a1 = posiciones_pulsos_abp[num_abp][0] - pos_i_abp
    a2 = posiciones_pulsos_abp[num_abp][1] - pos_i_abp
    plt.figure(1)
    plt.plot(range(p1,p2),y1[p1:p2])
    plt.title('Fotoplestimografía')
    plt.xlabel('Muestras')
    plt.ylabel('Amplitud(V)')
    plt.figure(2)
    plt.plot(range(a1,a2),signal_abp[a1:a2])
    plt.title('Presión Arterial')
    plt.xlabel('Muestras')
    plt.ylabel('Amplitud(mmHg)')

def get_dataset(hyperparameters,data):
  dataset=data
  features=pd.DataFrame()
  if (hyperparameters['PWD']):
    features=features.append(dataset['PWD'])
  if (hyperparameters['RatioSPDDP']):
    features=features.append(dataset['RatioSPDDP'])
  if (hyperparameters['Ratio BA']):
    features=features.append(dataset['Ratio BA'])
  if (hyperparameters['Sys_phase']):
    features=features.append(dataset['Sys_phase'])
  if (hyperparameters['Energia FFT']):
    features=features.append(dataset['Energia FFT'])
  if (hyperparameters['Kurtosis FFT']):
    features=features.append(dataset['Kurtosis FFT'])
  if (hyperparameters['Age']):
    features=features.append(dataset['Age'])
  if (hyperparameters['Gender']):
    features=features.append(dataset['Gender'])
  if (hyperparameters['Diast_phase']):
    features=features.append(dataset['Diast_phase'])
  return features

def huber_loss_wrapper(**huber_loss_kwargs):
    def huber_loss_wrapped_function(y_true, y_pred):
        return huber_loss(y_true, y_pred, **huber_loss_kwargs)
    return huber_loss_wrapped_function

def quantile_loss(q, y_p, y):
        e = y_p-y
        return tf.keras.backend.mean(tf.keras.backend.maximum(q*e, (q-1)*e))
def make_nn(hyperparameters,features):
  model=Sequential()
  n_cols=features.shape[1]#como la cantidad de features que tiene va variando en cada vez es necesario redefinir la mlp
  model.add(Dense(30, activation='elu',input_shape=(n_cols,),kernel_initializer='he_uniform'))
  # ,kernel_initializer='he_uniform'
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.3))
  model.add(Dense(10, activation='elu'))
  model.add(Dropout(rate=0.5))
  model.add(BatchNormalization())
  model.add(Dense(5, activation='elu'))
  model.add(Dropout(rate=0.5))
  model.add(BatchNormalization())
  model.add(Dense(2,activation='linear'))
  #model.compile(loss=huber_loss_wrapper(delta=0.5), optimizer= 'Adam', metrics=['mean_absolute_error'])
  
  quantile = 0.5
  model.compile(optimizer='adam', loss=lambda y_p, y: quantile_loss(quantile, y_p, y), metrics=['mean_absolute_error'])
  
  model.summary()
  return model

from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import mean_absolute_error
def make_score_classifier(hyperparameters,data):
  mae=[]
  for index, hp in hyperparameters.iterrows():
    print(hp)

    features=get_dataset(hp,data)
    features=np.transpose(features)
    bp=data[['Sistolica','Diastolica']]
    
    x,x_test,y,y_test=train_test_split(features, bp,test_size=0.1, random_state=42)
    x_train,x_val,y_train,y_val=train_test_split(x, y,test_size=0.1, random_state=42)

    media=x_train.mean()
    std=x_train.std()
    #if 'Gender' in features.columns:
    #  media['Gender']=0
    #  std['Gender']=1   
    x_train_norm = (x_train - media) / std #estandarizamos los datos
    x_val_norm= (x_val-x_val.mean())/x_val.std()

    model=make_nn(hp,features)
    # model.summary()
    hist=model.fit(x_train_norm, round(y_train), batch_size=1024, epochs=200, verbose=2, shuffle=True, validation_data=(x_val_norm,round(y_val)),callbacks=[stopper,CLR])
    y_pred=model.predict(x_val)
    mean_mae=mean_absolute_error(y_val, y_pred)
    mae.append(mean_mae)
  return acc

def evaluate_hyperparameters(hyperparameters, dataset):
    to = time.time()
    scores = make_score_classifier(hyperparameters,dataset)
    tf = time.time()
    print('finished in', (int(tf-to)), 'seconds.')

    return scores

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

def huber_loss_wrapper(**huber_loss_kwargs):
    def huber_loss_wrapped_function(y_true, y_pred):
        return huber_loss(y_true, y_pred, **huber_loss_kwargs)
    return huber_loss_wrapped_function

def quantile_loss(q, y_p, y):
        e = y_p-y
        return tf.keras.backend.mean(tf.keras.backend.maximum(q*e, (q-1)*e))


def select_features(X_train, y_train, X_test):
	fs = SelectKBest(score_func=f_regression, k='all')
	fs.fit(X_train, y_train)
	X_train_fs = fs.transform(X_train)
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

def res_block_first(input_layer,filters=16):
    
    #Primera capa
    
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(input_layer)
    aux_net=BatchNormalization()(net)
    
    #conexión directa
    
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(aux_net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(net)
    net=BatchNormalization()(net)
    
    #Shortcut
    net=Add()([aux_net,net])
    
    net=Activation("relu")(net)
    net=MaxPooling1D()(net)
    
    return net

def res_block(input_layer,filters=16):
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(input_layer)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(net)
    net=BatchNormalization()(net)
    
    #Shortcut
    net=Add()([input_layer,net])
    net=Activation("relu")(net)
    
    return net

def res_block_dec(input_layer,filters=32):
    
    input_layer=MaxPooling1D()(input_layer)
    net=Conv1D(filters=filters, kernel_size=1, padding='same')(input_layer)
    aux_net=BatchNormalization()(net)
    
    #conexión directa
    
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(input_layer)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv1D(filters=filters, kernel_size=3, padding='same')(net)
    net=BatchNormalization()(net)
    net=Add()([aux_net,net])
    net=Activation("relu")(net)
    
    return net



filename = '/content/drive/My Drive/Proyecto Final/Clasificadores de Señal/PPG/RL_PPG_250_kFFT_k_skewlibro.sav'
# load the model from disk
loaded_model_ppg = pickle.load(open(filename, 'rb'))

filename = '/content/drive/My Drive/Proyecto Final/Clasificadores de Señal/ABP/RL_ABP_125_kFFT_k.sav'
# load the model from disk
loaded_model_abp = pickle.load(open(filename, 'rb'))

FileNotFoundError: ignored

# Seleccion de Pacientes Random

In [ ]:
patients_141 = pd.read_csv('/content/drive/My Drive/Proyecto Final/150patients/abp_cnn_600_150patients', index_col=0)
patients_141

,Sistolica,Diastolica,PatientID,IcuStay
0,155.715004,60.493423,p098514,269040
1,153.785662,61.690104,p098514,269040
2,153.565864,61.348195,p098514,269040
3,153.590286,60.859754,p098514,269040
4,154.616012,60.859754,p098514,269040
...,...,...,...,...
595,143.333024,71.581035,p048212,288547
596,148.339545,72.509073,p048212,288547
597,145.335632,72.313696,p048212,288547
598,140.548910,71.410080,p048212,288547


Hay menos ICU stays (estadias en UTI) que pacientes porque cada paciente puede haber sido ingresado mas de una vez.

In [ ]:
icu_141 = np.unique(patients_141['IcuStay'].values)
len(icu_141)

141

En el dataset entero, hay 2509 ICU stays unicos:

In [ ]:
print(len(np.unique(datasetampliado['1'].values)))
icu_unicos = np.unique(datasetampliado['1'].values)

2509


In [ ]:
hola = datasetampliado[datasetampliado['1'].isin(icu_unicos)]
h = hola[['0','1']]
h
h = h.drop_duplicates(subset = '1')
h = h.reset_index(drop=True)
pd.DataFrame(h).to_csv('/content/drive/My Drive/Proyecto Final/Lista_2509_ICUSTAYS.csv')

In [ ]:
print(len(np.unique(datasetampliado['0'].values)))

2362


In [ ]:
import random
# n = random.randint(0,22)

posiciones = []

posiciones = random.sample(range(0,2509),9)

icu_pos = np.concatenate([icu_141 , icu_unicos[posiciones]])
print(len(icu_pos))
print(len(np.unique(icu_pos)))
icu_pos

150
150


array([200050, 200330, 200477, 200732, 200773, 201818, 201973, 202142,
       202293, 205056, 205981, 206008, 206113, 206610, 208973, 210700,
       210908, 212629, 212928, 213937, 215896, 216117, 216570, 217015,
       217070, 217702, 218387, 218888, 220693, 222348, 222464, 223592,
       224603, 224711, 224886, 224970, 225469, 226006, 227048, 227347,
       228189, 228297, 228694, 228838, 229293, 229896, 230001, 230135,
       230717, 231846, 232321, 233778, 233981, 234520, 234678, 234813,
       235249, 236628, 236762, 237444, 238940, 239100, 240176, 240536,
       240835, 241261, 241308, 242681, 243173, 244469, 249608, 249718,
       251990, 252005, 252340, 252895, 253584, 255308, 256387, 257626,
       258909, 259220, 260302, 260450, 260581, 260687, 260779, 260952,
       261497, 263127, 263404, 264937, 264947, 265589, 265615, 265726,
       266784, 267203, 268097, 269040, 269109, 269930, 269946, 270418,
       273148, 273149, 273351, 273484, 273917, 274052, 274664, 275794,
      

In [ ]:
hola = datasetampliado[datasetampliado['1'].isin(icu_pos)]
pacientes_150 = hola.drop_duplicates(subset = '0')[['0','1']]
print(len(pacientes_150['1'].values))
pacientes_150 = pacientes_150.reset_index(drop =  True)
pacientes_150 = pacientes_150.rename(columns = {'0': 'PatientID'}, inplace=False)
pacientes_150 = pacientes_150.rename(columns = {'1': 'IcuStay'}, inplace=False)
pacientes_150

149


,PatientID,IcuStay
0,p004324,227347
1,p005742,260450
2,p007184,290526
3,p008318,279287
4,p008698,256387
...,...,...
144,p094216,282606
145,p094669,233778
146,p099556,224970
147,p099669,229293


In [ ]:
datasetampliado.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376
0,p002906,237626,0.515152,0.518084,0.521017,0.523949,0.526882,0.530792,0.533724,0.535679,0.536657,0.536657,0.537634,0.539589,0.541544,0.542522,0.544477,0.547410,0.548387,0.550342,0.551320,0.552297,0.553275,0.553275,0.552297,0.551320,0.548387,0.546432,0.544477,0.543500,0.542522,0.540567,0.539589,0.539589,0.538612,0.537634,0.537634,0.537634,0.537634,0.537634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,p002906,237626,0.489736,0.491691,0.495601,0.499511,0.504399,0.508309,0.512219,0.516129,0.520039,0.522972,0.521017,0.518084,0.517107,0.518084,0.520039,0.521017,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,p002906,237626,0.446725,0.447703,0.448680,0.448680,0.448680,0.448680,0.449658,0.451613,0.453568,0.457478,0.462366,0.467253,0.471163,0.476051,0.480938,0.485826,0.489736,0.492669,0.494624,0.495601,0.495601,0.495601,0.495601,0.496579,0.498534,0.499511,0.501466,0.503421,0.504399,0.505376,0.506354,0.506354,0.508309,0.510264,0.510264,0.511241,0.511241,0.511241,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,p002906,237626,0.450635,0.452590,0.453568,0.454545,0.457478,0.458456,0.461388,0.464321,0.467253,0.470186,0.473118,0.475073,0.478983,0.483871,0.488759,0.493646,0.497556,0.500489,0.503421,0.505376,0.508309,0.511241,0.514174,0.517107,0.519062,0.521017,0.521017,0.521994,0.521994,0.521994,0.521994,0.521994,0.526882,0.533724,0.537634,0.539589,0.540567,0.541544,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,p002906,237626,0.494624,0.497556,0.500489,0.503421,0.506354,0.509286,0.514174,0.518084,0.521017,0.524927,0.528837,0.531769,0.535679,0.538612,0.539589,0.541544,0.542522,0.543500,0.544477,0.546432,0.548387,0.551320,0.552297,0.553275,0.553275,0.552297,0.550342,0.547410,0.545455,0.543500,0.542522,0.541544,0.541544,0.540567,0.538612,0.537634,0.536657,0.536657,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data = []
for icu in icu_pos: 
  patient_id = datasetampliado['0'].loc[datasetampliado['1']==icu].values[0]
  data.append([patient_id, icu] )


Se asocia un patient ID a cada ICU stay:

In [ ]:
df = pd.DataFrame(data)
df = df.rename(columns={0: 'PatientID', 1: 'IcuStay'}, inplace=False)
df

,PatientID,IcuStay
0,p078716,200050
1,p088202,200330
2,p094117,200477
3,p029961,200732
4,p044788,200773
...,...,...
145,p013719,283238
146,p056307,286167
147,p054987,282098
148,p063992,204344


In [ ]:
pd.DataFrame(df).to_csv('/content/drive/My Drive/Proyecto Final/Lista_150_CAPITULO1.csv')

## Lista de 50 pacientes

In [ ]:
icu_pos = [224216, 264697, 226712, 265868, 237826, 264400, 274107, 239657,
       228514, 297819, 255308, 262022, 277786, 229138, 245127, 262203,
       269109, 228119, 240435, 295187, 265287, 205429, 219451, 246367,
       239550, 221468, 255099, 292635, 261204, 292809, 242986,
       284076, 279857, 204737, 275839, 232166, 239136, 293373, 265337,
       256500, 222218, 288947, 264508, 249143, 202519, 211419,
       273861, 268715, 203383, 221215]

In [ ]:
icu_last_recorded = 268715
index_icu = icu_pos.index(icu_last_recorded)+1
icu_pos = icu_pos[index_icu:index_icu+6]
icu_pos

[203383, 221215]

In [ ]:
icu_pos.pop(2)

221468

In [ ]:
icu_pos

[249143, 202519, 211419, 273861, 268715]

# Resultados para 120 pulsos

## Dataset 120 - modificado con PatientID y ICUID

In [ ]:
ppg0=pd.read_csv('/content/drive/My Drive/Proyecto Final/ppg_cnn_120_0a4_id.csv',sep=',')
abp0=pd.read_csv('/content/drive/My Drive/Proyecto Final/abp_cnn_120_0a4_id.csv',sep=',')
ppg1=pd.read_csv('/content/drive/My Drive/Proyecto Final/ppg_cnn_120_5a9_id.csv',sep=',')
abp1=pd.read_csv('/content/drive/My Drive/Proyecto Final/abp_cnn_120_5a9_id.csv',sep=',')
ppg0=ppg0.drop(['Unnamed: 0'], axis = 1)
abp0=abp0.drop(['Unnamed: 0'], axis = 1)
ppg1=ppg1.drop(['Unnamed: 0'], axis = 1)
abp1=abp1.drop(['Unnamed: 0'], axis = 1)
parte0=ppg0.append(ppg1)
parte0abp=np.transpose(abp0).append([np.transpose(abp1)])
datasetampliado=parte0
datasetampliadoabp=np.transpose(parte0abp)
datasetampliado

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376
0,p002906,237626,0.515152,0.518084,0.521017,0.523949,0.526882,0.530792,0.533724,0.535679,0.536657,0.536657,0.537634,0.539589,0.541544,0.542522,0.544477,0.547410,0.548387,0.550342,0.551320,0.552297,0.553275,0.553275,0.552297,0.551320,0.548387,0.546432,0.544477,0.543500,0.542522,0.540567,0.539589,0.539589,0.538612,0.537634,0.537634,0.537634,0.537634,0.537634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,p002906,237626,0.489736,0.491691,0.495601,0.499511,0.504399,0.508309,0.512219,0.516129,0.520039,0.522972,0.521017,0.518084,0.517107,0.518084,0.520039,0.521017,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,0.521994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,p002906,237626,0.446725,0.447703,0.448680,0.448680,0.448680,0.448680,0.449658,0.451613,0.453568,0.457478,0.462366,0.467253,0.471163,0.476051,0.480938,0.485826,0.489736,0.492669,0.494624,0.495601,0.495601,0.495601,0.495601,0.496579,0.498534,0.499511,0.501466,0.503421,0.504399,0.505376,0.506354,0.506354,0.508309,0.510264,0.510264,0.511241,0.511241,0.511241,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,p002906,237626,0.450635,0.452590,0.453568,0.454545,0.457478,0.458456,0.461388,0.464321,0.467253,0.470186,0.473118,0.475073,0.478983,0.483871,0.488759,0.493646,0.497556,0.500489,0.503421,0.505376,0.508309,0.511241,0.514174,0.517107,0.519062,0.521017,0.521017,0.521994,0.521994,0.521994,0.521994,0.521994,0.526882,0.533724,0.537634,0.539589,0.540567,0.541544,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,p002906,237626,0.494624,0.497556,0.500489,0.503421,0.506354,0.509286,0.514174,0.518084,0.521017,0.524927,0.528837,0.531769,0.535679,0.538612,0.539589,0.541544,0.542522,0.543500,0.544477,0.546432,0.548387,0.551320,0.552297,0.553275,0.553275,0.552297,0.550342,0.547410,0.545455,0.543500,0.542522,0.541544,0.541544,0.540567,0.538612,0.537634,0.536657,0.536657,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198645,p099982,275958,0.318671,0.335288,0.358749,0.388074,0.422287,0.461388,0.502444,0.544477,0.585533,0.623656,0.659824,0.687195,0.708700,0.729228,0.746823,0.760508,0.770283,0.775171,0.776149,0.772239,0.765396,0.755621,0.741935,0.725318,0.707722,0.690127,0.669599,0.647116,0.623656,0.599218,0.575758,0.552297,0.529814,0.509286,0.489736,0.472141,0.458456,0.447703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198646,p099982,275958,0.324536,0.348974,0.379277,0.413490,0.450635,0.487781,0.524927,0.561095,0.589443,0.612903,0.637341,0.658847,0.677419,0.691105,0.700880,0.705767,0.706745,0.702835,0.695992,0.685239,0.672532,0.657869,0.641251,0.620723,0.592375,0.55914

In [ ]:
datasetampliado.loc[datasetampliado['1'] == icu_pos[0], '0'].values[0]

'p044735'

In [ ]:
datasetampliado['1'].isin(icu_pos)
hola = datasetampliado[datasetampliado['1'].isin(icu_pos)]

In [ ]:
np.unique(hola['0'].values)

'p002639', 'p004331', 'p011431', 'p012878', 'p014328', 'p027554',
       'p030222', 'p030575', 'p031290', 'p042310', 'p044735', 'p044806',
       'p045138', 'p045492', 'p045918', 'p046611', 'p048011', 'p049261',
       'p050336', 'p051724', 'p056490', 'p058286', 'p059828', 'p062647',
       'p062674', 'p062824', 'p063402', 'p064008', 'p064839', 'p068919',
       'p069296', 'p072048', 'p072377', 'p072439', 'p079396', 'p079664',
       'p080497', 'p081449', 'p081633', 'p083752', 'p085235', 'p087310',
       'p088206', 'p088907', 'p089437', 'p090605', 'p092651', 'p093360',
       'p094837', 'p096928'

array(['p002639', 'p004331', 'p011431', 'p012878', 'p014328', 'p027554',
       'p030222', 'p030575', 'p031290', 'p042310', 'p044735', 'p044806',
       'p045138', 'p045492', 'p045918', 'p046611', 'p048011', 'p049261',
       'p050336', 'p051724', 'p056490', 'p058286', 'p059828', 'p062647',
       'p062674', 'p062824', 'p063402', 'p064008', 'p064839', 'p068919',
       'p069296', 'p072048', 'p072377', 'p072439', 'p079396', 'p079664',
       'p080497', 'p081449', 'p081633', 'p083752', 'p085235', 'p087310',
       'p088206', 'p088907', 'p089437', 'p090605', 'p092651', 'p093360',
       'p094837', 'p096928'], dtype=object)

In [ ]:
np.unique(hola['1'].values)

array([202519, 203383, 204737, 205429, 211419, 219451, 221215, 221468,
       222218, 224216, 226712, 228119, 228514, 229138, 232166, 237826,
       239136, 239550, 239657, 240435, 242986, 245127, 246367, 249143,
       255099, 255308, 256500, 261204, 262022, 262203, 264400, 264508,
       264697, 265287, 265337, 265868, 268715, 269109, 273861, 274107,
       275839, 277786, 279857, 284076, 288947, 292635, 292809, 293373,
       295187, 297819])

In [ ]:
hola[['0','1']]

,0,1
1440,p004331,265287
1441,p004331,265287
1442,p004331,265287
1443,p004331,265287
1444,p004331,265287
...,...,...
191895,p094837,237826
191896,p094837,237826
191897,p094837,237826
191898,p094837,237826


In [ ]:
pacientes_50_def = hola.drop_duplicates(subset = '0')[['0','1']]
pacientes_50_def = pacientes_50_def.reset_index(drop=True)

pd.DataFrame(pacientes_50_def).to_csv('/content/drive/My Drive/Proyecto Final/Lista_50_CAPITULO1.csv')

## Red CNN+MLP - 50 Pacientes - Cross Validation

La idea es agarrar de a un paciente y ver si funciona bien la red para predecir sus datos

Elijo el primer ICUstay_ID

In [ ]:
datasetampliadoabp = datasetampliadoabp.reset_index(drop=True)
datasetampliadoabp_t = np.transpose(datasetampliadoabp)
datasetampliadoabp_t = datasetampliadoabp_t.reset_index(drop=True)

scores = []

ppg_120  = datasetampliado.loc[datasetampliado['1'].isin(icu_pos)]
icu_pos_str = icu_pos  
icu_pos_str = list(map(str,icu_pos))
abp_120 = datasetampliadoabp_t.loc[datasetampliadoabp_t[1].isin(icu_pos_str)]

medias = []
desvios = []
icu_utilizados = []
MAEs, LOSSs = [], []

for paciente in icu_pos:

  print(paciente)
  x = ppg_120.loc[ppg_120['1'].isin([paciente])]

  if (x.shape[0]<120):
    continue

  x = x.drop(['0','1'], axis=1)
  paciente_str = str(paciente)
  y = abp_120.loc[abp_120[1].isin([paciente_str])]
  y = y.drop([0,1], axis=1)
  y[2] = pd.to_numeric(y[2], downcast='float')
  y[3] = pd.to_numeric(y[3], downcast='float')
  x = np.transpose(np.transpose(x).head(300))

  x = x.reset_index(drop=True)
  y = y.reset_index(drop=True)

  #RED

  seed = 7
  np.random.seed(seed)

  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=20, min_lr=0.0000001,verbose=False)
  stopper=EarlyStopping(monitor="val_loss",patience=300,verbose=True,restore_best_weights=True)

  kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
  cvscores = []
  histories = []

  for train, test in kfold.split(x, y):

    media_x = x.mean(axis=1)
    desvio_x = x.std(axis=1)

    x_norm = x.fillna(0)
    x_norm = (x_norm - media_x) / desvio_x

    x_train = x.iloc[train] #ya estan normalizados
    x_val = x.iloc[test]

    break

    x_train_resh = np.array(x_train).reshape(-1,300,1)
    x_val_resh = np.array(x_val).reshape(-1,300,1)
    
    model = Sequential()
    model.add(Conv1D(filters=16,kernel_size=3,padding='same',activation='relu',input_shape=(300,1)))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=64,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))

    model.add(BatchNormalization())

    model.add(Flatten())

    model.add(Dense(150, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(20,activation='linear'))

    model.add(Dense(2, activation='linear'))#input: 512 #output:100 
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['mean_absolute_error'])
    
    hist=model.fit(x_train_resh, y.iloc[train], batch_size=128, epochs=2000, verbose=0,shuffle=True,validation_data=(x_val_resh,y.iloc[test]),callbacks=[stopper,reduce_lr])
    
    # evaluate the model
    score = model.evaluate(x_val_resh, y.iloc[test])
    print(score)

    histories.append(hist)
    cvscores.append(score)

    val_pred = model.predict(x_val_resh)

    e=np.abs(val_pred-y.iloc[test])
    mean_e=np.mean(e)
    desvio_e=np.std(e)

    print('La media del error abs es:' + str(mean_e))
    print('El desvio del error abs es:' + str(desvio_e))

    icu_utilizados.append(paciente)
    medias.append(mean_e)
    desvios.append(desvio_e)

    # scores.append(cvscores)  

  for i in cvscores:
      MAEs.append(i[0])
      LOSSs.append(i[1])

media_s, media_d = [], []
for i in medias:
    media_s.append(i[2])
    media_d.append(i[3])

desvio_s, desvio_d = [], []
for i in desvios:
    desvio_s.append(i[2])
    desvio_d.append(i[3])

d = {'ICU': icu_utilizados, 'media_s': media_s, 'desvio_s': desvio_s, 'media_d': media_d, 'desvio_d': desvio_d, 'loss': LOSSs, 'MAE': MAEs}
df = pd.DataFrame(data=d)


df.to_csv('/content/drive/My Drive/Proyecto Final/Resultados_50_120.csv')

224216
264697
226712
265868
237826
264400
274107
239657
228514
297819
255308
262022
277786
229138
245127
262203
269109
228119
240435
295187
265287
205429
219451
246367
239550
221468
255099
292635
261204
292809
242986
284076
279857
204737
275839
232166
239136
293373
265337
256500
222218
288947
264508
249143
202519
211419
273861
268715
203383
221215


In [ ]:
print(x_norm)
print(x_train)
np.mean(x_train, axis = 1)

      0   1   2   3   4   5   6   7   8  ...  91  92  93  94  95  96  97  98  99
0   NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN
1   NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN
2   NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN
3   NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN
4   NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..
115 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN
116 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN
117 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN
118 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN
119 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN

[120 rows x 420 columns]
  

0      0.445683
1      0.614995
2      0.626921
3      0.453956
4      0.394086
         ...   
115    0.451955
116    0.458169
117    0.513144
118    0.509293
119    0.495347
Length: 108, dtype: float64

In [ ]:
MAEs, LOSSs = [], []
for c in scores:
  for i in c:
    MAEs.append(c[0])
    LOSSs.append(c[1])

media_s, media_d = [], []
for i in medias:
    media_s.append(i[2])
    media_d.append(i[3])

desvio_s, desvio_d = [], []
for i in desvios:
    desvio_s.append(i[2])
    desvio_d.append(i[3])

d = {'ICU': icu_utilizados, 'media_s': media_s, 'desvio_s': desvio_s, 'media_d': media_d, 'desvio_d': desvio_d, 'loss': LOSSs, 'MAE': MAEs}
df = pd.DataFrame(data=d)

df.to_csv('/content/drive/My Drive/Proyecto Final/Resultados_50_120.csv')

ValueError: ignored

In [ ]:
scores

[[[11521.6044921875, 101.8302993774414],
  [11614.4951171875, 101.78719329833984],
  [59.1192741394043, 6.233661651611328],
  [59.3084716796875, 5.427834987640381],
  [8658.3681640625, 87.3244400024414],
  [10780.75, 98.1814193725586],
  [70.72206115722656, 6.874331951141357],
  [40.90353775024414, 5.122275352478027],
  [10212.7705078125, 95.77762603759766],
  [11092.0078125, 99.8206787109375]],
 [[11521.6044921875, 101.8302993774414],
  [11614.4951171875, 101.78719329833984],
  [59.1192741394043, 6.233661651611328],
  [59.3084716796875, 5.427834987640381],
  [8658.3681640625, 87.3244400024414],
  [10780.75, 98.1814193725586],
  [70.72206115722656, 6.874331951141357],
  [40.90353775024414, 5.122275352478027],
  [10212.7705078125, 95.77762603759766],
  [11092.0078125, 99.8206787109375]],
 [[11521.6044921875, 101.8302993774414],
  [11614.4951171875, 101.78719329833984],
  [59.1192741394043, 6.233661651611328],
  [59.3084716796875, 5.427834987640381],
  [8658.3681640625, 87.3244400024414]

# Resultados para 400 pulsos

## Red CNN+MLP - 50 Pacientes - Cross Validation

In [ ]:
ppg_400=pd.read_csv('/content/drive/My Drive/Proyecto Final/50patients/ppg_cnn_400_50patients.csv',sep=',')
ppg_400 = ppg_400.drop(columns = ['Unnamed: 0'])
ppg_400.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376
0,p014328,224216,0.332356,0.352884,0.375367,0.398827,0.423265,0.443793,0.462366,0.521994,0.602151,0.651026,0.684262,0.705767,0.719453,0.727273,0.731183,0.729228,0.724340,0.715543,0.703812,0.688172,0.670577,0.650049,0.627566,0.606061,0.584555,0.565005,0.546432,0.529814,0.515152,0.502444,0.491691,0.483871,0.475073,0.468231,0.460411,0.453568,0.445748,0.437928,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,p014328,224216,0.340176,0.369501,0.400782,0.431085,0.458456,0.482893,0.501466,0.514174,0.523949,0.531769,0.535679,0.537634,0.538612,0.536657,0.533724,0.528837,0.521994,0.514174,0.505376,0.494624,0.483871,0.471163,0.457478,0.442815,0.429130,0.416422,0.403715,0.392962,0.383187,0.373412,0.365591,0.357771,0.351906,0.345064,0.338221,0.332356,0.328446,0.326491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,p014328,224216,0.289345,0.315738,0.344086,0.374389,0.404692,0.433040,0.459433,0.482893,0.502444,0.517107,0.526882,0.536657,0.543500,0.548387,0.551320,0.552297,0.550342,0.546432,0.539589,0.530792,0.520039,0.507331,0.494624,0.480938,0.465298,0.448680,0.431085,0.413490,0.396872,0.381232,0.368524,0.358749,0.350929,0.345064,0.340176,0.337243,0.334311,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,p014328,224216,0.242424,0.245357,0.250244,0.258065,0.270772,0.285435,0.303030,0.325513,0.352884,0.382209,0.413490,0.442815,0.470186,0.493646,0.517107,0.539589,0.553275,0.562072,0.566960,0.567937,0.567937,0.565005,0.561095,0.555230,0.547410,0.537634,0.526882,0.514174,0.500489,0.484848,0.471163,0.456500,0.441838,0.427175,0.413490,0.400782,0.390029,0.381232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,p014328,224216,0.304985,0.326491,0.350929,0.379277,0.404692,0.428152,0.454545,0.479961,0.504399,0.524927,0.541544,0.555230,0.565005,0.571848,0.575758,0.577713,0.576735,0.573803,0.568915,0.563050,0.555230,0.546432,0.535679,0.523949,0.510264,0.495601,0.478983,0.462366,0.446725,0.433040,0.420332,0.408602,0.396872,0.387097,0.379277,0.372434,0.367546,0.364614,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
abp_400=pd.read_csv('/content/drive/My Drive/Proyecto Final/50patients/abp_cnn_400_50patients.csv',sep=',')
abp_400 = abp_400.drop(columns = ['Unnamed: 0'])
abp_400 = np.transpose(abp_400)
abp_400.head()

,0,1,2,3
0,p014328,224216,91.91661793782131,65.90617073413998
0.1,p014328,224216,94.06785793211077,68.74189618115787
0.2,p014328,224216,93.57893975159044,68.74189618115787
0.3,p014328,224216,92.69888702665386,67.5684925479091
0.4,p014328,224216,94.16564156821484,67.76405982011721


In [ ]:
patients = pd.read_csv('/content/drive/My Drive/Proyecto Final/Lista_50_CAPITULO1.csv',sep=',')
patients = patients.drop(columns=['Unnamed: 0'])
patients.head()

,0,1
0,p004331,265287
1,p002639,228514
2,p011431,262203
3,p012878,297819
4,p014328,224216


In [ ]:
patients = patients.rename(columns = {'0':'PatientID', '1':'IcuStay'}, inplace = False)

In [ ]:
patients[40:50]

,PatientID,IcuStay
40,p083752,268715
41,p085235,262022
42,p081449,232166
43,p081633,284076
44,p087310,219451
45,p090605,264697
46,p092651,228119
47,p093360,226712
48,p096928,239550
49,p094837,237826


In [ ]:
from sklearn.model_selection import KFold
from keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
medias = []
desvios = []
icu_utilizados = []
MAEs, LOSSs = [], []
N = 400

for i, data in patients[40:50].iterrows():
  icu_stay = data['IcuStay']
  patient_id=data['PatientID']
  print(icu_stay)
  
  x = ppg_400.loc[ppg_400['1'].isin([icu_stay])]
  x = x.head(N)
  
  if (x.shape[0]<N):
    continue

  x = x.drop(['0','1'], axis=1)
  x = np.transpose(np.transpose(x).head(300))

  icu_stay = str(icu_stay)
  y = abp_400.loc[abp_400[1].isin([icu_stay])]
  y = y.drop([0,1], axis=1)
  y[2] = pd.to_numeric(y[2], downcast='float')
  y[3] = pd.to_numeric(y[3], downcast='float')
  y = y.head(N)
  

  x = x.reset_index(drop=True)
  y = y.reset_index(drop=True)

  #RED

  seed = 7
  np.random.seed(seed)

  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=20, min_lr=0.0000001,verbose=False)
  stopper=EarlyStopping(monitor="val_loss",patience=300,verbose=True,restore_best_weights=True)

  kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
  cvscores = []
  histories = []

  for train, test in kfold.split(x, y):

    media_x = x.mean(axis=1)
    desvio_x = x.std(axis=1)

    x_norm = x.fillna(0)
    x_norm = (x_norm - media_x) / desvio_x

    x_train = x.iloc[train] #ya estan normalizados
    x_val = x.iloc[test]

    x_train_resh = np.array(x_train).reshape(-1,300,1)
    x_val_resh = np.array(x_val).reshape(-1,300,1)
    
    model = Sequential()
    model.add(Conv1D(filters=16,kernel_size=3,padding='same',activation='relu',input_shape=(300,1)))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=64,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))

    model.add(BatchNormalization())

    model.add(Flatten())

    model.add(Dense(150, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(20,activation='linear'))

    model.add(Dense(2, activation='linear'))#input: 512 #output:100 
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['mean_absolute_error'])
    
    hist=model.fit(x_train_resh, y.iloc[train], batch_size=128, epochs=2000, verbose=0,shuffle=True,validation_data=(x_val_resh,y.iloc[test]),callbacks=[stopper,reduce_lr])
    
    # evaluate the model
    score = model.evaluate(x_val_resh, y.iloc[test])
    print(score)

    histories.append(hist)
    cvscores.append(score)

    val_pred = model.predict(x_val_resh)

    e=np.abs(val_pred-y.iloc[test])
    mean_e=np.mean(e)
    desvio_e=np.std(e)

    print('La media del error abs es:' + str(mean_e))
    print('El desvio del error abs es:' + str(desvio_e))

    icu_utilizados.append(icu_stay)
    medias.append(mean_e)
    desvios.append(desvio_e)

    # scores.append(cvscores)  

  for i in cvscores:
      MAEs.append(i[0])
      LOSSs.append(i[1])

media_s, media_d = [], []
for i in medias:
    media_s.append(i[2])
    media_d.append(i[3])

desvio_s, desvio_d = [], []
for i in desvios:
    desvio_s.append(i[2])
    desvio_d.append(i[3])

d = {'ICU': icu_utilizados, 'media_s': media_s, 'desvio_s': desvio_s, 'media_d': media_d, 'desvio_d': desvio_d, 'loss': LOSSs, 'MAE': MAEs}
df = pd.DataFrame(data=d)


df.to_csv('/content/drive/My Drive/Proyecto Final/Resultados_50_400_parte9.csv')

268715
Restoring model weights from the end of the best epoch.
Epoch 00468: early stopping
2/2 [==============================] - 0s 96ms/step - loss: 22.1233 - mean_absolute_error: 3.3442
[22.1232967376709, 3.344229221343994]
La media del error abs es:2    3.547719
3    3.140740
dtype: float32
El desvio del error abs es:2    2.131930
3    4.153422
dtype: float32
Restoring model weights from the end of the best epoch.
Epoch 00444: early stopping
2/2 [==============================] - 0s 8ms/step - loss: 25.2692 - mean_absolute_error: 3.3154
[25.269216537475586, 3.315415620803833]
La media del error abs es:2    3.495083
3    3.135748
dtype: float32
El desvio del error abs es:2    2.592715
3    4.665591
dtype: float32
Restoring model weights from the end of the best epoch.
Epoch 01048: early stopping
2/2 [==============================] - 0s 5ms/step - loss: 12.0298 - mean_absolute_error: 2.6028
[12.029826164245605, 2.6027884483337402]
La media del error abs es:2    2.851550
3    2.35402

# Resultados para 120 pulsos - Normalizando

## Red CNN+MLP - 50 Pacientes - Cross Validation - Normalizando

La idea es agarrar de a un paciente y ver si funciona bien la red para predecir sus datos

In [ ]:
icu_pos = [224216, 264697, 226712, 265868, 237826, 264400, 274107, 239657,
       228514, 297819, 255308, 262022, 277786, 229138, 245127, 262203,
       269109, 228119, 240435, 295187, 265287, 205429, 219451, 246367,
       239550, 221468, 255099, 292635, 261204, 292809, 242986,
       284076, 279857, 204737, 275839, 232166, 239136, 293373, 265337,
       256500, 222218, 288947, 264508, 249143, 202519, 211419,
       273861, 268715, 203383, 221215]

In [ ]:
icu_last_recorded = 202519
index_icu = icu_pos.index(icu_last_recorded)+1
icu_pos = icu_pos[index_icu:index_icu+5]
icu_pos

[211419, 273861, 268715, 203383, 221215]

Elijo el primer ICUstay_ID

In [ ]:
datasetampliadoabp = datasetampliadoabp.reset_index(drop=True)
datasetampliadoabp_t = np.transpose(datasetampliadoabp)
datasetampliadoabp_t = datasetampliadoabp_t.reset_index(drop=True)

scores = []

ppg_120  = datasetampliado.loc[datasetampliado['1'].isin(icu_pos)]
icu_pos_str = icu_pos  
icu_pos_str = list(map(str,icu_pos))
abp_120 = datasetampliadoabp_t.loc[datasetampliadoabp_t[1].isin(icu_pos_str)]

medias = []
desvios = []
icu_utilizados = []
MAEs, LOSSs = [], []

for paciente in icu_pos:

  print(paciente)
  x = ppg_120.loc[ppg_120['1'].isin([paciente])]

  if (x.shape[0]<120):
    continue

  x = x.drop(['0','1'], axis=1)
  paciente_str = str(paciente)
  y = abp_120.loc[abp_120[1].isin([paciente_str])]
  y = y.drop([0,1], axis=1)
  y[2] = pd.to_numeric(y[2], downcast='float')
  y[3] = pd.to_numeric(y[3], downcast='float')
  x = np.transpose(np.transpose(x).head(300))

  x = x.reset_index(drop=True)
  y = y.reset_index(drop=True)

  #RED

  seed = 7
  np.random.seed(seed)

  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=20, min_lr=0.0000001,verbose=False)
  stopper=EarlyStopping(monitor="val_loss",patience=300,verbose=True,restore_best_weights=True)

  kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
  cvscores = []
  histories = []

  for train, test in kfold.split(x, y):

    x_norm = x.fillna(0)
    x_norm = (x - x.mean(axis=1)[:,None]) / x.std(axis = 1)[:,None]

    x_train = x_norm.iloc[train] #ya estan normalizados
    x_val = x_norm.iloc[test]
    
    x_train_resh = np.array(x_train).reshape(-1,300,1)
    x_val_resh = np.array(x_val).reshape(-1,300,1)
    
    model = Sequential()
    model.add(Conv1D(filters=16,kernel_size=3,padding='same',activation='relu',input_shape=(300,1)))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=64,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))

    model.add(BatchNormalization())

    model.add(Flatten())

    model.add(Dense(150, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(20,activation='linear'))

    model.add(Dense(2, activation='linear'))#input: 512 #output:100 
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['mean_absolute_error'])
    
    hist=model.fit(x_train_resh, y.iloc[train], batch_size=128, epochs=2000, verbose=0,shuffle=True,validation_data=(x_val_resh,y.iloc[test]),callbacks=[stopper,reduce_lr])
    
    # evaluate the model
    score = model.evaluate(x_val_resh, y.iloc[test])
    print(score)

    histories.append(hist)
    cvscores.append(score)

    val_pred = model.predict(x_val_resh)

    e=np.abs(val_pred-y.iloc[test])
    mean_e=np.mean(e)
    desvio_e=np.std(e)

    print('La media del error abs es:' + str(mean_e))
    print('El desvio del error abs es:' + str(desvio_e))

    icu_utilizados.append(paciente)
    medias.append(mean_e)
    desvios.append(desvio_e)

    # scores.append(cvscores)  

  for i in cvscores:
      MAEs.append(i[0])
      LOSSs.append(i[1])

media_s, media_d = [], []
for i in medias:
    media_s.append(i[2])
    media_d.append(i[3])

desvio_s, desvio_d = [], []
for i in desvios:
    desvio_s.append(i[2])
    desvio_d.append(i[3])

d = {'ICU': icu_utilizados, 'media_s': media_s, 'desvio_s': desvio_s, 'media_d': media_d, 'desvio_d': desvio_d, 'loss': LOSSs, 'MAE': MAEs}
df = pd.DataFrame(data=d)

df.to_csv('/content/drive/My Drive/Proyecto Final/Resultados_50_120.csv')

211419


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 01098: early stopping
1/1 [==============================] - 0s 28ms/step - loss: 36.6801 - mean_absolute_error: 3.9099
[36.6800537109375, 3.909886121749878]
La media del error abs es:2    4.860230
3    2.959542
dtype: float32
El desvio del error abs es:2    3.662711
3    5.250136
dtype: float32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00857: early stopping
1/1 [==============================] - 0s 28ms/step - loss: 169.3450 - mean_absolute_error: 6.2327
[169.34495544433594, 6.232655048370361]
La media del error abs es:2    9.628617
3    2.836692
dtype: float32
El desvio del error abs es:2    15.230624
3     2.441488
dtype: float32


...

Restoring model weights from the end of the best epoch.
Epoch 00612: early stopping
1/1 [==============================] - 0s 29ms/step - loss: 57.1467 - mean_absolute_error: 7.0392
[57.1467399597168, 7.0391764640808105]
La media del error abs es:2    5.563929
3    8.514423
dtype: float32
El desvio del error abs es:2    2.631056
3    1.979472
dtype: float32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00750: early stopping
1/1 [==============================] - 0s 27ms/step - loss: 88.3688 - mean_absolute_error: 7.2083
[88.3687973022461, 7.208276271820068]
La media del error abs es:2    9.062055
3    5.354498
dtype: float32
El desvio del error abs es:2    7.706136
3    2.561563
dtype: float32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00754: early stopping
1/1 [==============================] - 0s 26ms/step - loss: 10.0119 - mean_absolute_error: 2.6158
[10.011913299560547, 2.6158440113067627]
La media del error abs es:2    2.934788
3    2.296901
dtype: float32
El desvio del error abs es:2    1.899109
3    1.590120
dtype: float32


# Resultados para 200 pulsos - Normalizando

## Red CNN+MLP - 50 Pacientes - Cross Validation

In [ ]:
ppg_400=pd.read_csv('/content/drive/My Drive/Proyecto Final/50patients/ppg_cnn_400_50patients.csv',sep=',')
ppg_400 = ppg_400.drop(columns = ['Unnamed: 0'])
ppg_400.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376
0,p014328,224216,0.332356,0.352884,0.375367,0.398827,0.423265,0.443793,0.462366,0.521994,0.602151,0.651026,0.684262,0.705767,0.719453,0.727273,0.731183,0.729228,0.724340,0.715543,0.703812,0.688172,0.670577,0.650049,0.627566,0.606061,0.584555,0.565005,0.546432,0.529814,0.515152,0.502444,0.491691,0.483871,0.475073,0.468231,0.460411,0.453568,0.445748,0.437928,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,p014328,224216,0.340176,0.369501,0.400782,0.431085,0.458456,0.482893,0.501466,0.514174,0.523949,0.531769,0.535679,0.537634,0.538612,0.536657,0.533724,0.528837,0.521994,0.514174,0.505376,0.494624,0.483871,0.471163,0.457478,0.442815,0.429130,0.416422,0.403715,0.392962,0.383187,0.373412,0.365591,0.357771,0.351906,0.345064,0.338221,0.332356,0.328446,0.326491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,p014328,224216,0.289345,0.315738,0.344086,0.374389,0.404692,0.433040,0.459433,0.482893,0.502444,0.517107,0.526882,0.536657,0.543500,0.548387,0.551320,0.552297,0.550342,0.546432,0.539589,0.530792,0.520039,0.507331,0.494624,0.480938,0.465298,0.448680,0.431085,0.413490,0.396872,0.381232,0.368524,0.358749,0.350929,0.345064,0.340176,0.337243,0.334311,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,p014328,224216,0.242424,0.245357,0.250244,0.258065,0.270772,0.285435,0.303030,0.325513,0.352884,0.382209,0.413490,0.442815,0.470186,0.493646,0.517107,0.539589,0.553275,0.562072,0.566960,0.567937,0.567937,0.565005,0.561095,0.555230,0.547410,0.537634,0.526882,0.514174,0.500489,0.484848,0.471163,0.456500,0.441838,0.427175,0.413490,0.400782,0.390029,0.381232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,p014328,224216,0.304985,0.326491,0.350929,0.379277,0.404692,0.428152,0.454545,0.479961,0.504399,0.524927,0.541544,0.555230,0.565005,0.571848,0.575758,0.577713,0.576735,0.573803,0.568915,0.563050,0.555230,0.546432,0.535679,0.523949,0.510264,0.495601,0.478983,0.462366,0.446725,0.433040,0.420332,0.408602,0.396872,0.387097,0.379277,0.372434,0.367546,0.364614,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
abp_400=pd.read_csv('/content/drive/My Drive/Proyecto Final/50patients/abp_cnn_400_50patients.csv',sep=',')
abp_400 = abp_400.drop(columns = ['Unnamed: 0'])
abp_400 = np.transpose(abp_400)
abp_400.head()

,0,1,2,3
0,p014328,224216,91.91661793782131,65.90617073413998
0.1,p014328,224216,94.06785793211077,68.74189618115787
0.2,p014328,224216,93.57893975159044,68.74189618115787
0.3,p014328,224216,92.69888702665386,67.5684925479091
0.4,p014328,224216,94.16564156821484,67.76405982011721


In [ ]:
patients = pd.read_csv('/content/drive/My Drive/Proyecto Final/Lista_50_CAPITULO1.csv',sep=',')
patients = patients.drop(columns=['Unnamed: 0'])
patients.head()

,0,1
0,p004331,265287
1,p002639,228514
2,p011431,262203
3,p012878,297819
4,p014328,224216


In [ ]:
patients = patients.rename(columns = {'0':'PatientID', '1':'IcuStay'}, inplace = False)

In [ ]:
patients[30:50]

,PatientID,IcuStay
30,p068919,229138
31,p072048,239136
32,p072377,295187
33,p072439,261204
34,p079396,202519
35,p079664,205429
36,p080497,264508
37,p088206,256500
38,p088907,274107
39,p089437,265868


In [ ]:
from sklearn.model_selection import KFold
from keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
medias = []
desvios = []
icu_utilizados = []
MAEs, LOSSs = [], []
N = 200

for i, data in patients[30:50].iterrows():
  icu_stay = data['IcuStay']
  patient_id=data['PatientID']
  print(icu_stay)
  
  x = ppg_400.loc[ppg_400['1'].isin([icu_stay])]
  x = x.head(N)
  
  if (x.shape[0]<N):
    continue

  x = x.drop(['0','1'], axis=1)
  x = np.transpose(np.transpose(x).head(300))

  icu_stay = str(icu_stay)
  y = abp_400.loc[abp_400[1].isin([icu_stay])]
  y = y.drop([0,1], axis=1)
  y[2] = pd.to_numeric(y[2], downcast='float')
  y[3] = pd.to_numeric(y[3], downcast='float')
  y = y.head(N)
  

  x = x.reset_index(drop=True)
  y = y.reset_index(drop=True)

  #RED

  seed = 7
  np.random.seed(seed)

  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=20, min_lr=0.0000001,verbose=False)
  stopper=EarlyStopping(monitor="val_loss",patience=300,verbose=True,restore_best_weights=True)

  kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
  cvscores = []
  histories = []

  for train, test in kfold.split(x, y):

    x_norm = x.fillna(0)
    x_norm = (x - x.mean(axis=1)[:,None]) / x.std(axis = 1)[:,None]

    x_train = x_norm.iloc[train] #ya estan normalizados
    x_val = x_norm.iloc[test]
    
    x_train_resh = np.array(x_train).reshape(-1,300,1)
    x_val_resh = np.array(x_val).reshape(-1,300,1)
    
    model = Sequential()
    model.add(Conv1D(filters=16,kernel_size=3,padding='same',activation='relu',input_shape=(300,1)))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=64,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))

    model.add(BatchNormalization())

    model.add(Flatten())

    model.add(Dense(150, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(20,activation='linear'))

    model.add(Dense(2, activation='linear'))#input: 512 #output:100 
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['mean_absolute_error'])
    
    hist=model.fit(x_train_resh, y.iloc[train], batch_size=128, epochs=2000, verbose=0,shuffle=True,validation_data=(x_val_resh,y.iloc[test]),callbacks=[stopper,reduce_lr])
    
    # evaluate the model
    score = model.evaluate(x_val_resh, y.iloc[test])
    print(score)

    histories.append(hist)
    cvscores.append(score)

    val_pred = model.predict(x_val_resh)

    e=np.abs(val_pred-y.iloc[test])
    mean_e=np.mean(e)
    desvio_e=np.std(e)

    print('La media del error abs es:' + str(mean_e))
    print('El desvio del error abs es:' + str(desvio_e))

    icu_utilizados.append(icu_stay)
    medias.append(mean_e)
    desvios.append(desvio_e)

    # scores.append(cvscores)  

  for i in cvscores:
      MAEs.append(i[0])
      LOSSs.append(i[1])

media_s, media_d = [], []
for i in medias:
    media_s.append(i[2])
    media_d.append(i[3])

desvio_s, desvio_d = [], []
for i in desvios:
    desvio_s.append(i[2])
    desvio_d.append(i[3])

d = {'ICU': icu_utilizados, 'media_s': media_s, 'desvio_s': desvio_s, 'media_d': media_d, 'desvio_d': desvio_d, 'loss': LOSSs, 'MAE': MAEs}
df = pd.DataFrame(data=d)


df.to_csv('/content/drive/My Drive/Proyecto Final/Resultados_50_200_norm_parte2.csv')

229138


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00707: early stopping
1/1 [==============================] - 0s 26ms/step - loss: 4.4318 - mean_absolute_error: 1.5156
[4.431790351867676, 1.5155640840530396]
La media del error abs es:2    2.400120
3    0.631008
dtype: float32
El desvio del error abs es:2    1.595114
3    0.400555
dtype: float32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00948: early stopping
1/1 [==============================] - 0s 19ms/step - loss: 12.3594 - mean_absolute_error: 2.7347
[12.359394073486328, 2.7346794605255127]
La media del error abs es:2    4.561013
3    0.908345
dtype: float32
El desvio del error abs es:2    1.687322
3    0.493759
dtype: float32


...

Restoring model weights from the end of the best epoch.
Epoch 00510: early stopping
1/1 [==============================] - 0s 18ms/step - loss: 27.2760 - mean_absolute_error: 3.5435
[27.27602767944336, 3.5434863567352295]
La media del error abs es:2    3.260291
3    3.826682
dtype: float32
El desvio del error abs es:2    2.907235
3    4.563666
dtype: float32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00512: early stopping
1/1 [==============================] - 0s 19ms/step - loss: 29.3865 - mean_absolute_error: 4.4224
[29.38654136657715, 4.422397136688232]
La media del error abs es:2    3.347356
3    5.497439
dtype: float32
El desvio del error abs es:2    2.887902
3    3.001079
dtype: float32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00500: early stopping
1/1 [==============================] - 0s 22ms/step - loss: 56.9950 - mean_absolute_error: 5.3232
[56.99497604370117, 5.32322883605957]
La media del error abs es:2    5.024741
3    5.621716
dtype: float32
El desvio del error abs es:2    5.009410
3    5.660746
dtype: float32


# Resultados para 400 pulsos - Normalizando

## Red CNN+MLP - 50 Pacientes - Cross Validation

In [ ]:
ppg_400=pd.read_csv('/content/drive/My Drive/ProyectoFinal/ppg_cnn_400_50patients.csv',sep=',')
ppg_400 = ppg_400.drop(columns = ['Unnamed: 0'])
ppg_400.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376
0,p014328,224216,0.332356,0.352884,0.375367,0.398827,0.423265,0.443793,0.462366,0.521994,0.602151,0.651026,0.684262,0.705767,0.719453,0.727273,0.731183,0.729228,0.724340,0.715543,0.703812,0.688172,0.670577,0.650049,0.627566,0.606061,0.584555,0.565005,0.546432,0.529814,0.515152,0.502444,0.491691,0.483871,0.475073,0.468231,0.460411,0.453568,0.445748,0.437928,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,p014328,224216,0.340176,0.369501,0.400782,0.431085,0.458456,0.482893,0.501466,0.514174,0.523949,0.531769,0.535679,0.537634,0.538612,0.536657,0.533724,0.528837,0.521994,0.514174,0.505376,0.494624,0.483871,0.471163,0.457478,0.442815,0.429130,0.416422,0.403715,0.392962,0.383187,0.373412,0.365591,0.357771,0.351906,0.345064,0.338221,0.332356,0.328446,0.326491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,p014328,224216,0.289345,0.315738,0.344086,0.374389,0.404692,0.433040,0.459433,0.482893,0.502444,0.517107,0.526882,0.536657,0.543500,0.548387,0.551320,0.552297,0.550342,0.546432,0.539589,0.530792,0.520039,0.507331,0.494624,0.480938,0.465298,0.448680,0.431085,0.413490,0.396872,0.381232,0.368524,0.358749,0.350929,0.345064,0.340176,0.337243,0.334311,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,p014328,224216,0.242424,0.245357,0.250244,0.258065,0.270772,0.285435,0.303030,0.325513,0.352884,0.382209,0.413490,0.442815,0.470186,0.493646,0.517107,0.539589,0.553275,0.562072,0.566960,0.567937,0.567937,0.565005,0.561095,0.555230,0.547410,0.537634,0.526882,0.514174,0.500489,0.484848,0.471163,0.456500,0.441838,0.427175,0.413490,0.400782,0.390029,0.381232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,p014328,224216,0.304985,0.326491,0.350929,0.379277,0.404692,0.428152,0.454545,0.479961,0.504399,0.524927,0.541544,0.555230,0.565005,0.571848,0.575758,0.577713,0.576735,0.573803,0.568915,0.563050,0.555230,0.546432,0.535679,0.523949,0.510264,0.495601,0.478983,0.462366,0.446725,0.433040,0.420332,0.408602,0.396872,0.387097,0.379277,0.372434,0.367546,0.364614,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
abp_400=pd.read_csv('/content/drive/My Drive/ProyectoFinal/abp_cnn_400_50patients.csv',sep=',')
abp_400 = abp_400.drop(columns = ['Unnamed: 0'])
abp_400 = np.transpose(abp_400)
abp_400.head()

,0,1,2,3
0,p014328,224216,91.91661793782131,65.90617073413998
0.1,p014328,224216,94.06785793211077,68.74189618115787
0.2,p014328,224216,93.57893975159044,68.74189618115787
0.3,p014328,224216,92.69888702665386,67.5684925479091
0.4,p014328,224216,94.16564156821484,67.76405982011721


In [ ]:
patients = pd.read_csv('/content/drive/My Drive/ProyectoFinal/Lista_50_CAPITULO1.csv',sep=',')
patients = patients.drop(columns=['Unnamed: 0'])
patients.head()

,0,1
0,p004331,265287
1,p002639,228514
2,p011431,262203
3,p012878,297819
4,p014328,224216


In [ ]:
patients = patients.rename(columns = {'0':'PatientID', '1':'IcuStay'}, inplace = False)

In [ ]:
patients[40:50]

,PatientID,IcuStay
40,p083752,268715
41,p085235,262022
42,p081449,232166
43,p081633,284076
44,p087310,219451
45,p090605,264697
46,p092651,228119
47,p093360,226712
48,p096928,239550
49,p094837,237826


In [ ]:
from sklearn.model_selection import KFold
from keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
medias = []
desvios = []
icu_utilizados = []
MAEs, LOSSs = [], []
N = 400

for i, data in patients[40:50].iterrows():
  icu_stay = data['IcuStay']
  patient_id=data['PatientID']
  print(icu_stay)
  
  x = ppg_400.loc[ppg_400['1'].isin([icu_stay])]
  x = x.head(N)
  
  if (x.shape[0]<N):
    continue

  x = x.drop(['0','1'], axis=1)
  x = np.transpose(np.transpose(x).head(300))

  icu_stay = str(icu_stay)
  y = abp_400.loc[abp_400[1].isin([icu_stay])]
  y = y.drop([0,1], axis=1)
  y[2] = pd.to_numeric(y[2], downcast='float')
  y[3] = pd.to_numeric(y[3], downcast='float')
  y = y.head(N)
  

  x = x.reset_index(drop=True)
  y = y.reset_index(drop=True)

  #RED

  seed = 7
  np.random.seed(seed)

  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=20, min_lr=0.0000001,verbose=False)
  stopper=EarlyStopping(monitor="val_loss",patience=300,verbose=True,restore_best_weights=True)

  kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
  cvscores = []
  histories = []

  for train, test in kfold.split(x, y):

    x_norm = x.fillna(0)
    x_norm = (x - x.mean(axis=1)[:,None]) / x.std(axis = 1)[:,None]

    x_train = x_norm.iloc[train] #ya estan normalizados
    x_val = x_norm.iloc[test]
    
    x_train_resh = np.array(x_train).reshape(-1,300,1)
    x_val_resh = np.array(x_val).reshape(-1,300,1)
    
    model = Sequential()
    model.add(Conv1D(filters=16,kernel_size=3,padding='same',activation='relu',input_shape=(300,1)))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=64,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))
    model.add(BatchNormalization())

    model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2,strides=2))

    model.add(BatchNormalization())

    model.add(Flatten())

    model.add(Dense(150, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='elu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(20,activation='linear'))

    model.add(Dense(2, activation='linear'))#input: 512 #output:100 
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['mean_absolute_error'])
    
    hist=model.fit(x_train_resh, y.iloc[train], batch_size=128, epochs=2000, verbose=0,shuffle=True,validation_data=(x_val_resh,y.iloc[test]),callbacks=[stopper,reduce_lr])
    
    # evaluate the model
    score = model.evaluate(x_val_resh, y.iloc[test])
    print(score)

    histories.append(hist)
    cvscores.append(score)

    val_pred = model.predict(x_val_resh)

    e=np.abs(val_pred-y.iloc[test])
    mean_e=np.mean(e)
    desvio_e=np.std(e)

    print('La media del error abs es:' + str(mean_e))
    print('El desvio del error abs es:' + str(desvio_e))

    icu_utilizados.append(icu_stay)
    medias.append(mean_e)
    desvios.append(desvio_e)

    # scores.append(cvscores)  

  for i in cvscores:
      MAEs.append(i[0])
      LOSSs.append(i[1])

media_s, media_d = [], []
for i in medias:
    media_s.append(i[2])
    media_d.append(i[3])

desvio_s, desvio_d = [], []
for i in desvios:
    desvio_s.append(i[2])
    desvio_d.append(i[3])

d = {'ICU': icu_utilizados, 'media_s': media_s, 'desvio_s': desvio_s, 'media_d': media_d, 'desvio_d': desvio_d, 'loss': LOSSs, 'MAE': MAEs}
df = pd.DataFrame(data=d)


df.to_csv('/content/drive/My Drive/ProyectoFinal/Resultados_50_400_norm_parte2.csv')

268715


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00436: early stopping
2/2 [==============================] - 0s 7ms/step - loss: 21.4934 - mean_absolute_error: 3.1148
[21.49336051940918, 3.1147987842559814]
La media del error abs es:2    3.136328
3    3.093269
dtype: float32
El desvio del error abs es:2    2.445010
3    4.195686
dtype: float32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00483: early stopping
2/2 [==============================] - 0s 7ms/step - loss: 32.8009 - mean_absolute_error: 4.1856
[32.80094528198242, 4.185636043548584]
La media del error abs es:2    4.757506
3    3.613766
dtype: float32
El desvio del error abs es:2    2.950021
3    4.605008
dtype: float32


...

Restoring model weights from the end of the best epoch.
Epoch 00510: early stopping
2/2 [==============================] - 0s 8ms/step - loss: 39.3749 - mean_absolute_error: 5.0556
[39.3748893737793, 5.0556135177612305]
La media del error abs es:2    6.016386
3    4.094842
dtype: float32
El desvio del error abs es:2    4.038442
3    3.078335
dtype: float32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00435: early stopping
2/2 [==============================] - 0s 5ms/step - loss: 25.0963 - mean_absolute_error: 3.5164
[25.09625816345215, 3.5163733959198]
La media del error abs es:2    3.897235
3    3.135513
dtype: float32
El desvio del error abs es:2    4.238279
3    2.685076
dtype: float32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Restoring model weights from the end of the best epoch.
Epoch 00494: early stopping
2/2 [==============================] - 0s 8ms/step - loss: 31.7687 - mean_absolute_error: 3.6842
[31.768657684326172, 3.68418550491333]
La media del error abs es:2    3.864011
3    3.504361
dtype: float32
El desvio del error abs es:2    3.820719
3    4.661363
dtype: float32


# VERSION 2 - 200 No Norm

In [ ]:
import pandas as pd
patients = pd.read_csv('/content/drive/My Drive/Proyecto Final/Lista_50_CAPITULO1.csv',sep=',')
patients = patients.drop(columns=['Unnamed: 0'])
patients = patients.rename(columns = {'0':'PatientID', '1':'IcuStay'}, inplace = False)
patients.head()

,PatientID,IcuStay
0,p004331,265287
1,p002639,228514
2,p011431,262203
3,p012878,297819
4,p014328,224216


In [ ]:
ppg_400=pd.read_csv('/content/drive/My Drive/Proyecto Final/50patients/ppg_cnn_400_50patients.csv',sep=',')
ppg_400 = ppg_400.drop(columns = ['Unnamed: 0'])
print(f'the shape is {ppg_400.shape}')
ppg_400.head()

the shape is (19747, 377)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376
0,p014328,224216,0.332356,0.352884,0.375367,0.398827,0.423265,0.443793,0.462366,0.521994,0.602151,0.651026,0.684262,0.705767,0.719453,0.727273,0.731183,0.729228,0.724340,0.715543,0.703812,0.688172,0.670577,0.650049,0.627566,0.606061,0.584555,0.565005,0.546432,0.529814,0.515152,0.502444,0.491691,0.483871,0.475073,0.468231,0.460411,0.453568,0.445748,0.437928,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,p014328,224216,0.340176,0.369501,0.400782,0.431085,0.458456,0.482893,0.501466,0.514174,0.523949,0.531769,0.535679,0.537634,0.538612,0.536657,0.533724,0.528837,0.521994,0.514174,0.505376,0.494624,0.483871,0.471163,0.457478,0.442815,0.429130,0.416422,0.403715,0.392962,0.383187,0.373412,0.365591,0.357771,0.351906,0.345064,0.338221,0.332356,0.328446,0.326491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,p014328,224216,0.289345,0.315738,0.344086,0.374389,0.404692,0.433040,0.459433,0.482893,0.502444,0.517107,0.526882,0.536657,0.543500,0.548387,0.551320,0.552297,0.550342,0.546432,0.539589,0.530792,0.520039,0.507331,0.494624,0.480938,0.465298,0.448680,0.431085,0.413490,0.396872,0.381232,0.368524,0.358749,0.350929,0.345064,0.340176,0.337243,0.334311,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,p014328,224216,0.242424,0.245357,0.250244,0.258065,0.270772,0.285435,0.303030,0.325513,0.352884,0.382209,0.413490,0.442815,0.470186,0.493646,0.517107,0.539589,0.553275,0.562072,0.566960,0.567937,0.567937,0.565005,0.561095,0.555230,0.547410,0.537634,0.526882,0.514174,0.500489,0.484848,0.471163,0.456500,0.441838,0.427175,0.413490,0.400782,0.390029,0.381232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,p014328,224216,0.304985,0.326491,0.350929,0.379277,0.404692,0.428152,0.454545,0.479961,0.504399,0.524927,0.541544,0.555230,0.565005,0.571848,0.575758,0.577713,0.576735,0.573803,0.568915,0.563050,0.555230,0.546432,0.535679,0.523949,0.510264,0.495601,0.478983,0.462366,0.446725,0.433040,0.420332,0.408602,0.396872,0.387097,0.379277,0.372434,0.367546,0.364614,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np
abp_400=pd.read_csv('/content/drive/My Drive/Proyecto Final/50patients/abp_cnn_400_50patients.csv',sep=',')
abp_400 = abp_400.drop(columns = ['Unnamed: 0'])
abp_400 = np.transpose(abp_400)
abp_400.head()

,0,1,2,3
0,p014328,224216,91.91661793782131,65.90617073413998
0.1,p014328,224216,94.06785793211077,68.74189618115787
0.2,p014328,224216,93.57893975159044,68.74189618115787
0.3,p014328,224216,92.69888702665386,67.5684925479091
0.4,p014328,224216,94.16564156821484,67.76405982011721


In [ ]:
from sklearn.model_selection import KFold
from keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

from keras.layers import Dense, Flatten, Dropout,Activation,BatchNormalization, Convolution2D, MaxPooling2D,Conv1D,MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
def cnn_v2():
  model = Sequential()
  model.add(Conv1D(filters=16,kernel_size=3,padding='same',activation='relu',input_shape=(300,1)))
  model.add(BatchNormalization())

  model.add(Conv1D(filters=64,kernel_size=3,padding='same',activation='relu'))
  model.add(MaxPooling1D(pool_size=2,strides=2))

  model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
  model.add(MaxPooling1D(pool_size=2,strides=2))


  model.add(Conv1D(filters=128,kernel_size=3,padding='same',activation='relu'))
  model.add(MaxPooling1D(pool_size=2,strides=2))
  model.add(BatchNormalization())

  model.add(Flatten())

  model.add(Dense(150, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(20,activation='relu'))
  model.add(Dense(2, activation='linear'))#input: 512 #output:100 
  model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['mean_absolute_error'])

  model.summary
  return model

GENERALIZED

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
medias = []
desvios = []
icu_utilizados = []
MAEs, LOSSs = [], []
N = 200

for i, data in patients[40:50].iterrows():
  icu_stay = data['IcuStay']
  patient_id=data['PatientID']
  print(f'Processing ICUStay {icu_stay}...')
  
  ppg_patient = ppg_400.loc[ppg_400['1'].isin([icu_stay])]
  print(f'the shape of ppg_patient is {ppg_patient.shape}')

  x = ppg_patient[0:N]
  x = x.drop(['0','1'], axis=1)
  x = np.transpose(np.transpose(x).head(300))
  x = x.reset_index(drop=True)
  
  
  if (x.shape[0] < N):
    continue

  abp_patient = abp_400.loc[abp_400[1].isin([str(icu_stay)])]
  print(f'the shape of abp_patient is {abp_patient.shape}')
  
  
  y = abp_patient[0:N]
  y = y.drop([0,1], axis=1)
  y[2] = pd.to_numeric(y[2], downcast='float')
  y[3] = pd.to_numeric(y[3], downcast='float')
  y = y.reset_index(drop=True)
  y.head()

  #RED

  seed = 7
  np.random.seed(seed)

  reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 20, min_lr = 0.0000001, verbose = False)
  stopper = EarlyStopping(monitor = "val_loss", patience = 150, verbose = True, restore_best_weights = True)

  kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
  cvscores = []
  histories = []

  x_norm = x
  ########
  #verificamos que este bien la normalizacion
  print(f'La media de x_norm es {x_norm.mean(axis = 1)}')
  print(f'El desvío de x_norm es {x_norm.std(axis = 1)}')

  for train, val in kfold.split(x_norm, y):

    x_train = x_norm.iloc[train]
    x_val = x_norm.iloc[val]

    print(f'The shape of x_train is {x_train.shape}')
    print(f'The shape of x_val is {x_val.shape}')

    x_train_resh = np.array(x_train).reshape(-1,300,1)
    x_val_resh = np.array(x_val).reshape(-1,300,1)

    y_train = y.iloc[train]
    y_val = y.iloc[val]
    
    model_v2 = cnn_v2()
    hist = model_v2.fit(x_train_resh, y_train, batch_size=64, epochs=2000, verbose=0,shuffle=True,validation_data=(x_val_resh,y_val),callbacks=[stopper,reduce_lr])
    # evaluate the model
    score = model_v2.evaluate(x_val_resh, y_val)
    print(score)

    histories.append(hist)
    cvscores.append(score)

    val_pred = model_v2.predict(x_val_resh)

    e = np.abs(val_pred - y.iloc[val])
    mean_e = np.mean(e)
    desvio_e = np.std(e)

    print(f'La media del error abs es: {str(mean_e)}')
    print(f'El desvio del error abs es: {str(desvio_e)}')

    icu_utilizados.append(icu_stay)
    medias.append(mean_e)
    desvios.append(desvio_e)

    # scores.append(cvscores)  
    
  for i in cvscores:
      MAEs.append(i[0])
      LOSSs.append(i[1])

media_s, media_d = [], []
for i in medias:
    media_s.append(i[2])
    media_d.append(i[3])

desvio_s, desvio_d = [], []
for i in desvios:
    desvio_s.append(i[2])
    desvio_d.append(i[3])

d = {'ICU': icu_utilizados, 'media_s': media_s, 'desvio_s': desvio_s, 'media_d': media_d, 'desvio_d': desvio_d, 'loss': LOSSs, 'MAE': MAEs}
df = pd.DataFrame(data=d)

df.to_csv('/content/drive/My Drive/Proyecto Final/Resultados_50_cnn_v2_200_no_norm_parte2.csv')

Processing ICUStay 268715...
the shape of ppg_patient is (400, 377)
the shape of abp_patient is (400, 4)
La media de x_norm es 0      0.535738
1      0.536201
2      0.545106
3      0.533356
4      0.502835
         ...   
195    0.484197
196    0.488824
197    0.543806
198    0.547775
199    0.510528
Length: 200, dtype: float64
El desvío de x_norm es 0      0.859950
1      0.907811
2      0.926068
3      0.909882
4      0.859541
         ...   
195    0.839597
196    0.879944
197    0.887042
198    0.929380
199    0.885130
Length: 200, dtype: float64
The shape of x_train is (180, 300)
The shape of x_val is (20, 300)
Restoring model weights from the end of the best epoch.
Epoch 00681: early stopping
1/1 [==============================] - 0s 17ms/step - loss: 7.4327 - mean_absolute_error: 2.2769
[7.432717800140381, 2.276890516281128]
La media del error abs es: 2    2.134116
3    2.419665
dtype: float32
El desvio del error abs es: 2    1.460644
3    1.524050
dtype: float32
The shape of x

# Dataset 400 pulsos 150 pacientes

In [ ]:
path_pulsos='/content/drive/My Drive/Proyecto Final/Pulsos'
path_abp='/content/drive/My Drive/Proyecto Final/Features'

import os
import csv
import pandas as pd
import numpy as np

ppg=pd.DataFrame()
abp=pd.DataFrame()
num_archivos=200
segs=125*3
for parte in range(0,1):
  parte='p0'+str(parte)
  path_parte=os.path.join(path_pulsos,parte)

  for patient in os.listdir(path_parte):

    if(patient == 'p002906_237626.csv'):
      path_file_ppg=os.path.join(path_parte,patient)
      path_file_abp=os.path.join(os.path.join(path_abp,parte),patient)

      id, stay_id=patient.split('_') 
      stay_id=stay_id.split('.csv')[0]

      archivo_abp=pd.read_csv(path_file_abp, sep=',', encoding='utf-8', quotechar='"', decimal='.', error_bad_lines=False)

      with open(path_file_ppg) as csv_file:
        file = csv.reader(csv_file, delimiter=',',)
        i=0
        for row in file:
          if (i<num_archivos):
            # ppg_signal=[[id],[stay_id],[float(i) for i in row]+list(np.zeros(segs-len(row)))]
            ppg_signal = [[id]+[stay_id]+[float(i) for i in row]+list(np.zeros(segs-len(row)))]
            ppg= ppg.append(ppg_signal)

            i+=1
          else:
            break

        j=0
        for idx,abp_signal in archivo_abp.iterrows():
          if (j<num_archivos):
            abp_signal=[id,stay_id]+list(abp_signal[['Sistolica','Diastolica']].values)
            abp=pd.concat([abp,pd.DataFrame(abp_signal)],axis=1)

            j+=1
          else:
            break
    else:
      break

# ppg.rename(index={0:'PatientID', 1:'ICU_Stay_ID'})
# abp.rename(index={0:'PatientID', 1:'ICU_Stay_ID',2:'Sistolica',3:'Diastolica'})


ppg.to_csv('/content/drive/My Drive/Proyecto Final/ppg_cnn_200_pat1.csv', sep=',', encoding='utf-8', quotechar='"', decimal='.')
abp.to_csv('/content/drive/My Drive/Proyecto Final/abp_cnn_200_pat1.csv', sep=',', encoding='utf-8', quotechar='"', decimal='.')